In [1]:
from pathlib import Path

WORKDIR = Path.cwd().absolute()

with (WORKDIR / "2015-09.txt").open("r") as file:
    inputs = [line.strip() for line in file.readlines()]

inputs[0:8]


['Faerun to Norrath = 129',
 'Faerun to Tristram = 58',
 'Faerun to AlphaCentauri = 13',
 'Faerun to Arbre = 24',
 'Faerun to Snowdin = 60',
 'Faerun to Tambi = 71',
 'Faerun to Straylight = 67',
 'Norrath to Tristram = 142']

In [2]:
from collections import defaultdict

distances = defaultdict(dict)

for line in inputs:
    tokens = line.split(" ")
    citya = tokens[0]
    cityb = tokens[2]
    distance = int(tokens[-1])
    distances[citya][cityb] = distance
    distances[cityb][citya] = distance


In [3]:
from typing import List, Set, Tuple
from collections import namedtuple

routes = set()

num_cities = len(distances.keys())

def closest(destinations: List, visited: Set=set()) -> Tuple[str, int]:

    flip = {distance:destination for destination, distance in destinations.items() if destination not in visited}
    distance = min(flip.keys())
    destination = flip[distance]

    return destination, distance

Route = namedtuple("Route", ["distance", "cities", "end", "count"])

# seed routes
for start, destinations in distances.items():
    destination, distance = closest(destinations)
    routes.add(Route(distance, f"{start}->{destination}", destination, 2))

while True:
    shortest = min((element.distance for element in routes))
    route = list(filter(lambda e: e.distance == shortest, routes))[0]
    if route.count == num_cities:
        print(f"Part 1: Shortest route is length {route.distance} for {route}")
        break
    next_city, distance = closest(distances[route.end], set(route.cities.split("->")))
    routes.remove(route)
    length = shortest + distance
    cities = f"{route.cities}->{next_city}"
    count = route.count + 1
    new_route = Route(length, cities, next_city, count)
    routes.add(Route(length, cities, next_city, count))

routes


Part 1: Shortest route is length 207 for Route(distance=207, cities='Tristram->Tambi->Snowdin->AlphaCentauri->Faerun->Arbre->Straylight->Norrath', end='Norrath', count=8)


{Route(distance=207, cities='Tristram->Tambi->Snowdin->AlphaCentauri->Faerun->Arbre->Straylight->Norrath', end='Norrath', count=8),
 Route(distance=210, cities='Arbre->Faerun->AlphaCentauri->Snowdin->Tambi->Tristram->Straylight', end='Straylight', count=7),
 Route(distance=213, cities='Norrath->AlphaCentauri->Snowdin->Tambi->Tristram->Faerun->Arbre->Straylight', end='Straylight', count=8),
 Route(distance=225, cities='Snowdin->AlphaCentauri->Faerun->Arbre->Straylight->Norrath->Tambi', end='Tambi', count=7),
 Route(distance=226, cities='Faerun->AlphaCentauri->Snowdin->Tambi->Tristram->Straylight->Arbre', end='Arbre', count=7),
 Route(distance=252, cities='AlphaCentauri->Snowdin->Tambi->Tristram->Faerun->Arbre->Straylight->Norrath', end='Norrath', count=8),
 Route(distance=295, cities='Straylight->Arbre->Faerun->AlphaCentauri->Snowdin->Tambi->Tristram->Norrath', end='Norrath', count=8),
 Route(distance=300, cities='Tambi->Snowdin->AlphaCentauri->Faerun->Arbre->Straylight->Norrath->Tristr

In [5]:
from itertools import permutations

def measure(path:List) -> int:

    length = 0
    for i in range(len(path)-1):
        length += distances[path[i]][path[i+1]]

    return length

longest = 0
path = []
for permutation in permutations(distances.keys()):
    length = measure(permutation)
    if length >longest:
        longest = length
        path = permutation

print(f"Part 2: Longest path is length {longest} - {path}")


Part 2: Longest path is length 804 - ('Tambi', 'Faerun', 'Norrath', 'Tristram', 'AlphaCentauri', 'Arbre', 'Snowdin', 'Straylight')
